In [4]:
!uv add langchain google-genai tiktoken rapidocr-onnxruntime python-dotenv langchain-community

Resolved 80 packages in 2.27s
Prepared 76 packages in 39.47s
Installed 76 packages in 10.50s
 + aiohappyeyeballs==2.6.1
 + aiohttp==3.13.3
 + aiosignal==1.4.0
 + annotated-types==0.7.0
 + anyio==4.12.1
 + attrs==25.4.0
 + certifi==2026.1.4
 + charset-normalizer==3.4.4
 + coloredlogs==15.0.1
 + dataclasses-json==0.6.7
 + distro==1.9.0
 + flatbuffers==25.12.19
 + frozenlist==1.8.0
 + google-auth==2.47.0
 + google-genai==1.59.0
 + greenlet==3.3.0
 + h11==0.16.0
 + httpcore==1.0.9
 + httpx==0.28.1
 + httpx-sse==0.4.3
 + humanfriendly==10.0
 + idna==3.11
 + jsonpatch==1.33
 + jsonpointer==3.0.0
 + langchain==1.2.6
 + langchain-classic==1.0.1
 + langchain-community==0.4.1
 + langchain-core==1.2.7
 + langchain-text-splitters==1.1.0
 + langgraph==1.0.6
 + langgraph-checkpoint==4.0.0
 + langgraph-prebuilt==1.0.6
 + langgraph-sdk==0.3.3
 + langsmith==0.6.4
 + marshmallow==3.26.2
 + mpmath==1.3.0
 + multidict==6.7.0
 + mypy-extensions==1.1.0
 + numpy==2.4.1
 + onnxruntime==1.23.2
 + opencv-python

In [4]:
import os

from dotenv import load_dotenv

load_dotenv()

os.environ["GEMINI_API_KEY"] = os.getenv("GEMINI_API_KEY")
# gemini_api_key = os.getenv("GEMINI_API_KEY")
# print(gemini_api_key)

## Data Ingestion


In [5]:
from langchain_community.document_loaders import TextLoader


In [6]:
loader = TextLoader(
    r"C:\Users\Abhinay\Desktop\LLMOps_rag\data\agentic_ai.txt",
    encoding="utf8"
)
document = loader.load()

In [7]:
print(document)

[Document(metadata={'source': 'C:\\Users\\Abhinay\\Desktop\\LLMOps_rag\\data\\agentic_ai.txt'}, page_content='What is Agentic AI?\nAgentic AI is an autonomous AI system that can act independently to achieve pre-determined goals. Traditional software follows pre-defined rules, and traditional artificial intelligence also requires prompting and step-by-step guidance. However, agentic AI is proactive and can perform complex tasks without constant human oversight. "Agentic" indicates agency — the ability of these systems to act independently, but in a goal-driven manner.\n\nAI agents can communicate with each other and other software systems to automate existing business processes. But beyond static automation, they make independent contextual decisions. They learn from their environment and adapt to changing conditions, enabling them to perform sophisticated workflows with accuracy.\n\nFor example, an agentic AI system can optimize employee shift schedules. If an employee is off sick, the

What is a Document in LangChain?

A LangChain Document looks like this:

Document(
    page_content="some text here",
    metadata={"source": "file.txt"}
)


So it has:

text → page_content

metadata → source, page number, URL, etc.

In [16]:
document[0].page_content[:50]  # Print the first 500 characters of the first documen

'What is Agentic AI?\nAgentic AI is an autonomous AI'

In [8]:
import tiktoken

text = document[0].page_content

# choose encoding based on model family
encoding = tiktoken.get_encoding("cl100k_base")  
# used by GPT-4, GPT-3.5, Gemini-style tokenization is close enough for budgeting

num_tokens = len(encoding.encode(text))
print(num_tokens)


3098


Core idea (1 sentence)

split_text works on raw strings
split_documents works on LangChain Document objects (and preserves metadata)


In [44]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=500, chunk_overlap=50,
    separators=["\n\n", "\n", ".", " "]
  )
texts_chunks = text_splitter.split_documents(document)

You’re asking:

Who adds this?

chunk.metadata = {"source": "agentic_ai.txt"}

Short answer

👉 LangChain adds it automatically, not you.

| Loader          | Metadata added                          |
| --------------- | --------------------------------------- |
| `TextLoader`    | `{"source": file_path}`                 |
| `PyPDFLoader`   | `{"source": file, "page": page_number}` |
| `WebBaseLoader` | `{"source": url}`                       |
| `CSVLoader`     | column / row info                       |


In [45]:
texts_chunks

[Document(metadata={'source': 'C:\\Users\\Abhinay\\Desktop\\LLMOps_rag\\data\\agentic_ai.txt'}, page_content='What is Agentic AI?\nAgentic AI is an autonomous AI system that can act independently to achieve pre-determined goals. Traditional software follows pre-defined rules, and traditional artificial intelligence also requires prompting and step-by-step guidance. However, agentic AI is proactive and can perform complex tasks without constant human oversight. "Agentic" indicates agency — the ability of these systems to act independently, but in a goal-driven manner.'),
 Document(metadata={'source': 'C:\\Users\\Abhinay\\Desktop\\LLMOps_rag\\data\\agentic_ai.txt'}, page_content='AI agents can communicate with each other and other software systems to automate existing business processes. But beyond static automation, they make independent contextual decisions. They learn from their environment and adapt to changing conditions, enabling them to perform sophisticated workflows with accurac

Rule (this is LLMOps 101):

If chunking logic changes → rebuild the vector index from scratch

👉 Each chunk is now its own Document object

note : u installed faiss-cpu using faiss

import faiss
This is the low-level FAISS library (Facebook AI Similarity Search).

FAISS does:

fast vector similarity search

nearest-neighbor lookup

cosine / L2 / inner-product search

LangChain does NOT replace FAISS — it wraps it.

In [10]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="nomic-embed-text",
    base_url="http://localhost:11434"
)

vector = embeddings.embed_query("hello, world!")
print(len(vector))
vector[:5]

C:\Users\Abhinay\AppData\Local\Temp\ipykernel_8504\1656819921.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(


768


[-0.041807375848293304,
 -0.11619870364665985,
 -4.202528476715088,
 -0.36897924542427063,
 -0.33735501766204834]

In [ ]:
# import faiss
# from langchain_community.docstore.in_memory import InMemoryDocstore
# from langchain_community.vectorstores import FAISS

# embedding_dim = len(embeddings.embed_query("hello world"))
# index = faiss.IndexFlatL2(embedding_dim)

# vector_store = FAISS(
#     embedding_function=embeddings,
#     index=index,
#     docstore=InMemoryDocstore(),
#     index_to_docstore_id={},
# )

No need to manually create: IndexFlatL2, InMemoryDocstore,index_to_docstore_id
LangChain handles it. this note is for below code 

In [47]:
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(
    texts_chunks,
    embedding=embeddings
)


above lines :Creates a FAISS index
Embeds texts_chunks
Stores them in the vector DB
So after this runs, your FAISS store is ready to query.

In [48]:
vector_store.index.ntotal


50

Mapping connects FAISS → Docstore

LangChain automatically updates:

index_to_docstore_id = {
  0: "uuid_1",
  1: "uuid_2",
  2: "uuid_3"
}


This is how:

FAISS result index=2

→ maps to document ID

→ fetches original text + metadata

In [ ]:
#buffer cell 

faiss.IndexFlatL2 → vector search engine

InMemoryDocstore → stores documents

embedding_function → how text becomes vectors

index_to_docstore_id → lookup table

FAISS(...) → binds everything into one retriever

In [49]:
vector_store

In [50]:
vector_store.save_local("vectorstore/faiss_index")


In [51]:
retriever=vector_store.as_retriever()

In [52]:
# Perform similarity search
query = "What is the Key Characteristics of Agentic AI?"
docs = vector_store.similarity_search(query, k=4)

# Display the results
for i, doc in enumerate(docs):
    print(f"Document {i+1}:")
    print(doc.page_content)
    print("-" * 50)
            

Document 1:
Adaptable
--------------------------------------------------
Document 2:
Trust and transparancy
--------------------------------------------------
Document 3:
What is Agentic AI?
Agentic AI is an autonomous AI system that can act independently to achieve pre-determined goals. Traditional software follows pre-defined rules, and traditional artificial intelligence also requires prompting and step-by-step guidance. However, agentic AI is proactive and can perform complex tasks without constant human oversight. "Agentic" indicates agency — the ability of these systems to act independently, but in a goal-driven manner.
--------------------------------------------------
Document 4:
Agentic AI enhances incident response speed while also providing a more specific and in-depth post-incident analysis to prevent the same errors from recurring in the future.
--------------------------------------------------


In [20]:
from langchain_core.prompts import ChatPromptTemplate

template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [21]:
prompt=ChatPromptTemplate.from_template(template)

In [22]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse ten sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:\n"), additional_kwargs={})])

In [25]:
from langchain_core.output_parsers import StrOutputParser

In [26]:
output_parser=StrOutputParser()

What StrOutputParser actually does (important)
This line: | StrOutputParser() means:
“Take the LLM output and return it as a plain Python string”
Without it, LangChain may return: AIMessage structured objects
metadata-wrapped responses , So this parser is required for clean RAG outputs.

In [38]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm_model = ChatGoogleGenerativeAI(
    model="models/gemini-2.5-flash",
    temperature=0.2,
    google_api_key=os.getenv("GEMINI_API_KEY"),
    
)


In [39]:
response = llm_model.invoke("Explain Agentic AI in one paragraph")
print(response.content)

Agentic AI refers to artificial intelligence systems designed to operate autonomously to achieve a specific, complex goal. Unlike traditional AI that simply responds to prompts, agentic AI actively perceives its environment, plans a sequence of actions, executes those actions (often by interacting with other tools, APIs, or the internet), and iteratively monitors its progress. It can break down large tasks into smaller sub-tasks, make decisions, learn from its outcomes, and adapt its strategy until the objective is met, essentially transforming a reactive AI into a proactive, goal-driven "agent" capable of independent problem-solving and task execution.


In [54]:
from langchain_core.runnables import RunnablePassthrough


rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm_model
    | output_parser
)

What counts as a Runnable?All of these are Runnables:
Component	Input → Output
PromptTemplate	dict → string
Retriever	string → documents
LLM	string → text
OutputParser	text → python object
Lambda	anything → anything

If it: accepts input , returns output , …it’s a Runnable.

The pipe operator (|) — the magic, This is runnable composition:

prompt | llm | StrOutputParser(), Which means:

Prompt gets input
Output goes to LLM
Output goes to parser
Equivalent to:parser(llm(prompt(input)))

In [55]:
rag_chain.invoke("tell me about Agentic AI")

'Agentic AI empowers employees to create lightweight AI applications that interact with common enterprise software and automate repetitive tasks. A key capability of AI agents is their perception stage, where they collect real-time data from diverse sources, including structured, semi-structured, and unstructured data. These agents directly interact with various APIs like RESTful, gRPC, and GraphQL to ingest necessary data from cloud platforms, enterprise systems, and SaaS applications. Amazon Q Business is a generative AI-powered assistant that leverages agentic AI, allowing users to find information, gain insights, and take action at work. Agentic AI is also characterized by being adaptable, with an emphasis on trust and transparency.'